### RBF Network Regression

In [5]:
import pandas as pd
from processing_utils import *
from RBF import RBF
from rbf_optimization import objective
import optuna
from optuna.trial import Trial
from sklearn.preprocessing import StandardScaler
%reload_ext autoreload
%autoreload 2

raw_dataset = pd.read_csv('./dataset/data.csv')

In [8]:
dataset = encode_smiles_column_of(
    prune_dataset_lines(
        raw_dataset,
        remove_nan_lines=False,
        remove_nan_cols=True,
        remove_duplicates=True
    ),
    strategy='count_encoding'
)


((val_f_scaler, val_t_scaler), (test_f_scaler, test_t_scaler)), (X_train, y_train, X_val, y_val, X_train_val, y_train_val, X_test, y_test) = scale_data(*get_train_data(
    dataset,
    targets_columns=['Energy_(kcal/mol)', 'Energy DG:kcal/mol)'],
    as_numpy=False
))


#### TODO : Use RBF.py to see performance.

In [9]:

study = optuna.create_study(
    direction='minimize',
    study_name="RBF hyperparameters optimization"
)

study.optimize(
    lambda trial: objective(trial, (X_train, y_train, X_val, y_val)),
    n_trials=20,
)

print(study.best_trial)

[I 2022-06-21 18:50:03,643] A new study created in memory with name: RBF hyperparameters optimization
[I 2022-06-21 18:50:10,752] Trial 0 finished with value: 1.0196089171818234 and parameters: {'n_clusters': 360, 'sigma': 1.7190362799760837}. Best is trial 0 with value: 1.0196089171818234.
[I 2022-06-21 18:50:15,165] Trial 1 finished with value: 1.012008619865041 and parameters: {'n_clusters': 220, 'sigma': 6.630825107754122}. Best is trial 1 with value: 1.012008619865041.
[I 2022-06-21 18:50:20,216] Trial 2 finished with value: 1.0169728727404568 and parameters: {'n_clusters': 262, 'sigma': 2.805776583645126}. Best is trial 1 with value: 1.012008619865041.
[I 2022-06-21 18:50:23,546] Trial 3 finished with value: 0.9921441092094803 and parameters: {'n_clusters': 145, 'sigma': 1.0572476130106636}. Best is trial 3 with value: 0.9921441092094803.
[I 2022-06-21 18:50:28,928] Trial 4 finished with value: 1.0219153364113451 and parameters: {'n_clusters': 299, 'sigma': 5.044753610084823}. Be

FrozenTrial(number=19, values=[0.9730791683001827], datetime_start=datetime.datetime(2022, 6, 21, 18, 51, 27, 296877), datetime_complete=datetime.datetime(2022, 6, 21, 18, 51, 29, 461537), params={'n_clusters': 10, 'sigma': 4.087362139173999}, distributions={'n_clusters': IntUniformDistribution(high=400, low=10, step=1), 'sigma': UniformDistribution(high=10.0, low=0.01)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=19, state=TrialState.COMPLETE, value=None)
